Read US Cities CSV file and put each city/state into a list

In [1]:
import pandas as pd

filename = 'uscities.csv'
cities = []

cities_df = pd.read_csv(filename)
for index,row in cities_df.iterrows():
    city = row.iloc[1]
    city = city.replace('St.','Saint')
    city = city.replace(' ','-')
    state = row.iloc[2]
    cities.append((city.strip(),state.strip()))

Create Dictionaries for Area Codes, Zip Codes, State Abbreviations, that will be used for webscraping and API calls

In [2]:
import pandas as pd

#initialize dictionaries
area_code_dict = dict()
state_code_dict = dict()
state_abbrevs_dict = dict()

#Create dictionary for area codes
area_code_df = pd.read_fwf('area_codes.txt')
for index, row in area_code_df.iterrows():
    area_code = row.values[0].split('\t')[1]
    city_and_state = row.values[0].split('\t')[2].split(',')
    if len(city_and_state) > 1:
        city = city_and_state[0]
        state_abbrev = city_and_state[1].strip()[:2]
        area_code_dict[str(area_code)] = (city,state_abbrev)

#Create dictionary for state codes
state_codes_df = pd.read_fwf('state_codes.txt',header=None)
for index, row in state_codes_df.iterrows():
    code, city_state = row.values
    state_code_dict[city_state] = str(code)

#Create dictionary for state codes
state_abbrevs_df = pd.read_csv('state_abbreviations.txt', sep='\t', header=None)
for index, row in state_abbrevs_df.iterrows():
    state, abbrev = row.values
    state_abbrevs_dict[abbrev] = state

Use the list of cities and states to webscrape the cost of living data and add it to the cost_of_living.csv

In [ ]:
import requests
import sys
from bs4 import BeautifulSoup
import csv

cost_living_csv = 'cost_of_living1002.csv'
header_made = 0

#Filter through the html to get the table data, then add it to the csv
def add_city_data(response,city_name,state):
    global header_made
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('table', class_='data_wide_table')
    if not table:
        sys.stderr.write(f'Cannot retreive data for {city_name}\n') 
        return None #end if table does not exist

    row_data = [city_name,state]
    header = ['City','State']

    for row in table.find_all('tr'):
        cols = row.find_all('td')
        data = [td.get_text(strip=True) for td in cols]
        if len(data) > 2:
            item = data[0].strip()
            avg_price = data[1].replace('\xa0$','').strip()
            price_range = data[2].strip()
            prices = (avg_price,price_range)
            row_data.append(prices)
            header.append(item)

    with open(cost_living_csv, 'a', newline = '') as file:
        writer = csv.writer(file)
        if not header_made:
            writer.writerow(header)
            header_made = 1
        writer.writerow(row_data)
        file.flush()
    
    return True

#Create URL from the city/state and retreive website response, call add_city_data function with response as an input
def fetch_data(city,state):
    city_name = city.replace(' ','-')
    url = f'https://www.numbeo.com/cost-of-living/in/{city_name}'
    backup_url = f'https://www.numbeo.com/cost-of-living/in/{city}-{state}-United-States'
    urls = [url,backup_url]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                result = add_city_data(response,city_name,state)
                if result: break
        except:
                sys.stderr.write(f'Cannot retreive data for {city_name}\n')

for city, state in cities:
    fetch_data(city,state)

Use the list of cities and states to webscrape the Quality of Life data and add it to the quality_of_life.csv

In [ ]:
import requests
import sys
from bs4 import BeautifulSoup
import csv

quality_life_csv = 'quality_of_life.csv'
header_made = 0
# cities = cities[101:]

#Filter through the html to get the table data, then add it to the csv
def add_quality_life_data(response,city_name,state):
    global header_made
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')
    table = tables[2]
    if not table: 
        sys.stderr.write(f'Cannot retreive data for {city_name}\n')
        return False#end if table does not exist

    row_data = [city_name,state]
    header = ['City','State']

    for row in table.find_all('tr'):
        cols = row.find_all('td')
        data = [td.get_text(strip=True) for td in cols]
        if len(data) > 2:
            item = data[0].strip()
            quality = data[1].strip()
            rating = data[2].strip()
            qualities = (quality,rating)
            row_data.append(qualities)
            header.append(item)
        

    with open(quality_life_csv, 'a', newline = '') as file:
        writer = csv.writer(file)
        if not header_made:
            writer.writerow(header)
            header_made = 1
        if len(row_data) > 2:    
            writer.writerow(row_data)
            file.flush()
    
    return True

#Create URL from the city/state and retreive website response, call add_quality_life_data function with response as an input
def fetch_quality_life_data(city,state):
    city_name = city.replace('-',' ')
    url = f'https://www.numbeo.com/quality-of-life/in/{city}'
    backup_url = f'https://www.numbeo.com/quality-of-life/in/{city}-{state}-United-States'
    urls = [url,backup_url]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                result = add_quality_life_data(response,city_name,state)
                if result: 
                    break
        except:
            sys.stderr.write(f'Cannot retreive data for {city_name}\n')

for city, state in cities:
    fetch_quality_life_data(city,state)

Write functions for making API calls to the Bureau of Labor Statistics server

In [3]:
import json
import requests
import sys

api_key = 'abbbbae8905543da8a40822dc27e7b88'
endpoint = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

def get_BLS_data(series_batch, **kwargs):
    #Pass BLS timeseries data to return data in JSON format, arguments can be added to specify search
    if len(series_batch) < 1 or len(series_batch) > 25:
        raise ValueError('SeriesId must be at least 1 and less than 25')
    
    series_ids = [item[0] for item in series_batch]
    headers = {'Content-Type': 'application/json'}
    payload = {
        'seriesid': series_ids,
        'registrationKey': api_key
    }
    
    #Update payload with keyword arguments and convert to JSON
    payload.update(kwargs)
    payload = json.dumps(payload)

    #Post request to BLS API
    response = requests.post(endpoint, data = payload, headers = headers)
    response.raise_for_status()

    #Return JSON response
    result = json.loads(response.text)
    if result['status'] != 'REQUEST_SUCCEEDED':
        raise Exception(result['message'])
    if result['status'] == 'REQUEST_SUCCEEDED':
        return result
    else:
        return None
    
def bundle(SeriesID_list, n=25):
    for i in range(0,len(SeriesID_list),n):
        yield SeriesID_list[i:i + n]


Build the Series IDs that will be used as an input to the functions above, and the city/state info to go along with it 

In [ ]:
import csv
import time 

#Initialize lists, iterators, static values
series_ids = []
input_data = []
n_api_calls = 0
failures = 0
measure_code = '06' #Labor Force Population
write_file = 'city_labor_force_population.csv'


#api key and endpoint for U.S. Bureau of Labor Statistics
api_key = 'abbbbae8905543da8a40822dc27e7b88'
endpoint = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

#After getting data from the API, add it to a csv
def add_BLS_data(data_value,tuple_data):
    seriesId,city_name,area_code,state_name,abbrev = tuple_data

    with open(write_file,'a',newline='') as file:
        writer = csv.writer(file)
        row = [data_value,city_name,area_code,state_name,abbrev]
        writer.writerow(row)
        file.flush()

#Build a list of Series IDs for the BLS API calls along with the city info in a tuple
for area_code,city_state in area_code_dict.items():
    city = city_state[0]
    state_abbrev = city_state[1]
    state_name = state_abbrevs_dict.get(state_abbrev,None)
    if not state_name:
        continue
    
    #Build the Series ID for the given zip code 
    seriesId = f'LAU{area_code}{measure_code}'
    city_info = (seriesId,city,area_code,state_name,state_abbrev)
    input_data.append(city_info)

for batch in bundle(input_data, 25):
    try:
        data = get_BLS_data(batch, startyear = 2024, endyear = 2024)
        series_list = data['Results']['series']

        for series, city_info in zip(series_list,batch):
            seriesId = city_info[0]
            data = series['data']
            if data:
                data_value = data[0]['value']
                add_BLS_data(data_value, city_info)
    except:
        failures += 1
        print(f' fail')

print(f'n: {n_api_calls}\n failures: {failures}')

Create function to add a header to a CSV File

In [4]:
import csv 
#Function to add a header to an existing CSV file
def add_header(filename,header):
    try:
        with open(filename,'r') as read_file:
            reader = csv.reader(read_file)
            rows = list(reader)

        with open(filename,'w', newline = '') as out_file:
            writer = csv.writer(out_file)
            writer.writerow(header)
            writer.writerows(rows)
    except:
        print(f'error writing to {filename}')

#Headers
header_unemployment_rate = ['Unemployment Rate','City','BLS Series ID','State','State Abbreviation']
header_labor_force_population = ['Labor Force Population','City','BLS Series ID','State','State Abbreviation']

#Files
unemployment_rate_file = 'city_unemployment_rate.csv'
labor_population_file = 'city_labor_force_population.csv'

# add_header(unemployment_rate_file,header_unemployment_rate)